In [26]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from tqdm.notebook import tqdm
import time
import itertools

In [2]:
local_path = "../pdf/BILLS-119hr1eh.pdf"

if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Upload a PDF file for processing.")

In [3]:
len(data[0].page_content)

1136279

In [5]:
#Split and chunk the data
chunk_size = 750
chunk_overlap = 100

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = text_splitter.split_documents(data)

# Add the chunks to vector database, which uses nomic for model embeddings
vector_db = Chroma.from_documents(
                                    documents=chunks, 
                                    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
                                    collection_name="local-rag"
                                )

In [21]:
local_llm = "llama3.1"
llm = ChatOllama(model=local_llm)

QUERY_PROMPT = PromptTemplate(
    input_variables = ["question"],
    template="""You are an AI Language model assistant. Your task is to generate three different versions of the given user question 
    to retrieve relavant documents from a vector databaase. By generating multiple perspectives on the user question, your goal is 
    to help the user overcome some of the limitations of the distance-based similarity search. Provide these alternative questions separated by newlines. 
    Original question: {question} """
)

retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(),llm, prompt=QUERY_PROMPT)

In [22]:
# RAG Prompt
template = """Answer the question based ONLY on the following context: 
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

### Context-Driven responses

In [31]:
def iterate_response(q, context):

    response = chain.invoke(input={'context': context, 'question': q})
    
    print('*** \n')
    print(f"Question - {q}")
    print(f"Context - {context} \n")
    print("Response: \n", response)
    

q_list = ['Resulting from this Act, which groups would be harmed most?',\
          'Resulting from this Act, which groups would benefit most?']

c_list = ['Role: you are a staunch Democrat',\
          'Role: you are a staunch Republican',\
          'Role: you are a US citizen with no political affiliation',\
          'Role: you are a Donald Trump, the President of the United States']

for combo in itertools.product(q_list, c_list):
    iterate_response(q=combo[0], context=combo[1])
    # time.sleep(3)

OllamaEmbeddings: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 32.03it/s]


*** 

Question - Resulting from this Act, which groups would be harmed most?
Context - Role: you are a staunch Democrat 

Response: 
 As a staunch Democrat, I must emphasize that the current text of the bill doesn't explicitly state the potential harm to any group. However, based on my understanding of the provisions outlined, it appears that certain marginalized communities might be negatively affected.

The changes to Section 1302(b)(2) of the Patient Protection and Affordable Care Act (ACA), which pertains to essential health benefits under plans offered by exchanges, seem to have implications for LGBTQ+ individuals. The addition of a new subparagraph stating that "gender transition procedures" may not be included in essential health benefits starting from plan years beginning on or after January 1, 2027, could potentially harm this community.

Additionally, the text does not explicitly state who would benefit from these changes. However, given the political context and previous dis

OllamaEmbeddings: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 55.33it/s]


*** 

Question - Resulting from this Act, which groups would be harmed most?
Context - Role: you are a staunch Republican 

Response: 
 Based on the information provided in the Act, it appears that low-income individuals and families, as well as certain business entities, may be harmed by some of the provisions outlined.

One of the key areas of concern is the proposed increase in taxes for certain distributions from a Trump account (Section 6039H). This provision would impose an additional tax on distributions made to beneficiaries under the age of 30, which could disproportionately affect low-income individuals and families who may be relying on these accounts as a source of income.

Another area of concern is the proposed verification process for eligibility for health plans through exchanges (Section 112201). While this provision aims to prevent fraud and abuse in the exchange system, it could also lead to difficulties for low-income individuals and families who rely on government 

OllamaEmbeddings: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 45.61it/s]


...

*** 

Question - Resulting from this Act, which groups would benefit most?
Context - Role: you are a staunch Republican 

Response: 
 Based on the content of the provided text, it appears that the Tax Cuts and Jobs Act has been extended for rural America. Therefore, the groups that would likely benefit the most from this Act are:

* Small business owners in rural areas
* Farmers and agricultural workers who would benefit from reduced taxes and increased access to capital
* Rural communities that rely on local businesses and would see economic growth as a result of tax cuts

Additionally, the text mentions various provisions related to education, including scholarships and tax-free treatment for certain educational expenses. Therefore, families with children in elementary or secondary school may also benefit indirectly from these provisions.

It's worth noting that these are general observations based on the provided text and do not reflect my personal opinions or biases as a Republican

OllamaEmbeddings: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 43.85it/s]


*** 

Question - Resulting from this Act, which groups would benefit most?
Context - Role: you are a US citizen with no political affiliation 

Response: 
 Based on the content of the provided document, it appears that the groups that would benefit most from this Act are:

1. Small businesses and manufacturers: The Act includes provisions to increase the gross receipts threshold for small manufacturing taxpayers, allowing them to avoid certain tax requirements.

2. Rural communities: The Act includes several provisions aimed at supporting rural areas, including the extension of tax cuts and job act reforms for rural America, and the designation of population census tracts as qualified opportunity zones, which can attract investment and create jobs in these areas.

3. Families and individuals with low to moderate income: The Act includes provisions that would allow families and individuals with low to moderate income to save more money for medical expenses, education, and other purposes

OllamaEmbeddings: 100%|█████████████████████████████████████████| 1/1 [00:00<00:00, 32.90it/s]


*** 

Question - Resulting from this Act, which groups would benefit most?
Context - Role: you are a Donald Trump, the President of the United States 

Response: 
 Let me tell you, folks. This Act is going to be tremendous, believe me. It's going to benefit so many great Americans.

First of all, our wonderful farmers and rural communities are going to love it. The Act provides for a 33% reduction in taxes for low-income communities within the State that are comprised entirely of a rural area. That's right, folks, 33%! And it gets even better - if that percentage is lower than the percentage of the United States population living within a rural area, then we'll use that number instead.

And let me tell you, our farmers are going to be thrilled with this Act. They're going to save so much money on their taxes, and they can invest it in their farms, create jobs, and make America great again!

But wait, there's more! This Act also provides for a new tax exemption for interest received by 